In [3]:
"""
This script will compare TWO excel files that have the same columns.
It will produce a third Excel file (let's call it the production file) that highlights the changed cells, and will produce a sheet that displays any new added rows

The only things that need to be changed are in:

Line 53: Change OUTPUT file path/directory.
Line 100: Old data file path
Line 101: New data file path
Line 111: The index column. This is the column that you want to use as an index, or the anchor which the code can use to compare the two files. 
Ideally, you want the index column to be something unique, like email address or student number, etc...

Points to note:
- Please keep the r' before the file path
- The DIFF sheet in the production file highlights the changed cells.
- There will be two sheets in the workbook, one has the old data and one has the new data
- There will also be an 'added' sheet. This shows only the new entries/rows

"""
import pandas as pd
from pathlib import Path


def excel_diff(path_OLD, path_NEW, index_col):

    df_OLD = pd.read_excel(path_OLD, index_col=index_col).fillna(0)
    df_NEW = pd.read_excel(path_NEW, index_col=index_col).fillna(0)

    # Perform Diff
    dfDiff = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    
    for row in dfDiff.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
                if value_OLD==value_NEW:
                    dfDiff.loc[row,col] = df_NEW.loc[row,col]
                else:
                    dfDiff.loc[row,col] = ('{}→{}').format(value_OLD,value_NEW)
        else:
            newRows.append(row)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            dfDiff = dfDiff.append(df_OLD.loc[row,:])

    dfDiff = dfDiff.sort_index().fillna('')
    print(dfDiff)
    print('\nNew Rows:     {}'.format(newRows))
    print('Dropped Rows: {}'.format(droppedRows))

    # Save output and format
    fname = r'C:\Users\USER1\Data\{} vs {}.xlsx'.format(path_OLD.stem,path_NEW.stem)
    writer = pd.ExcelWriter(fname, engine='xlsxwriter')

    dfDiff.to_excel(writer, sheet_name='DIFF', index=True)
    df_NEW.to_excel(writer, sheet_name=path_NEW.stem, index=True)
    df_OLD.to_excel(writer, sheet_name=path_OLD.stem, index=True)
    added = df_NEW.loc[newRows]

    # get xlsxwriter objects
    workbook  = writer.book
    worksheet = writer.sheets['DIFF']
    added.to_excel(writer, sheet_name='added', index=True)
    worksheet.hide_gridlines(2)
    worksheet.set_default_row(15)

    # define formats
    date_fmt = workbook.add_format({'align': 'center', 'num_format': 'yyyy-mm-dd'})
    center_fmt = workbook.add_format({'align': 'center'})
    number_fmt = workbook.add_format({'align': 'center', 'num_format': '#,##0.00'})
    cur_fmt = workbook.add_format({'align': 'center', 'num_format': '$#,##0.00'})
    perc_fmt = workbook.add_format({'align': 'center', 'num_format': '0%'})
    grey_fmt = workbook.add_format({'font_color': '#E0E0E0'})
    highlight_fmt = workbook.add_format({'font_color': '#FF0000', 'bg_color':'#B1B3B3'})
    new_fmt = workbook.add_format({'font_color': '#32CD32','bold':True})

    # set format over range
    ## highlight changed cells
    worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
                                            'criteria': 'containing',
                                            'value':'→',
                                            'format': highlight_fmt})

    # highlight new/changed rows
    for row in range(dfDiff.shape[0]):
        if row+1 in newRows:
            worksheet.set_row(row+1, 15, new_fmt)
        if row+1 in droppedRows:
            worksheet.set_row(row+1, 15, grey_fmt)

    # save
    writer.save()
    print('\nDone.\n')
    


def main():
    path_OLD = Path(r'C:\Users\USER1\Data\Data_Old.xlsx')
    path_NEW = Path(r'C:\Users\USER1\Data\Data_New.xlsx')

    # get index col from data
    df = pd.read_excel(path_NEW)
    index_col = 'stud_num'
    print('\nIndex column: {}\n'.format(index_col))

    excel_diff(path_OLD, path_NEW, index_col)


if __name__ == '__main__':
    main()



Index column: stud_num

           first_name last_name course pref_no accepted? sem_1_perc  \
stud_num                                                              
19284726          Ash        MA   B234       2         n         45   
957463803       Frank        PR   U887       1         n         45   
968462598     Anthony        AH   C450       3         y         90   
987654321         Sam        PE   C223       1         y         99   
1028445267      James        CD   C287       1         n         88   
1092746598       John        AB   C450       1         y         76   
1284756532     Daniel        LL   U887       3         n         86   
3029871654  Cassandra        CL   C230       3         y         36   
3049876946        Jim        MK   B337     5→6         n         48   
3099875509    Tiffany        DO   L998       2         n     39→100   
3784710098    Melissa        LO   B377       4         y         87   
9476251984       Tina        KL  H2039       2      